In [1]:
import json
import pandas as pd
from copy import deepcopy

In [2]:
import os
os.chdir("../")

In [3]:
source = []
with open("assets/data/rag_truth/source_info.jsonl", "r") as f:
    for line in f:
        source.append(json.loads(line))

In [4]:
id_to_source = {elem.pop("source_id") : elem for elem in deepcopy(source)}

In [7]:
response = []
with open("assets/data/rag_truth/response.jsonl", "r") as f:
    for line in f:
        response.append(json.loads(line))

In [8]:
models = set([elem["model"] for elem in response])

In [9]:
models

{'gpt-3.5-turbo-0613',
 'gpt-4-0613',
 'llama-2-13b-chat',
 'llama-2-70b-chat',
 'llama-2-7b-chat',
 'mistral-7B-instruct'}

In [10]:
sources_summary_cnn = [elem for elem in source if elem["task_type"] == "Summary" and elem["source"] == "CNN/DM"]
summary_cnn_ids = [elem["source_id"] for elem in sources_summary_cnn]

In [11]:
sources_qa = [elem for elem in source if elem["task_type"] == "QA"]
qa_ids = [elem["source_id"] for elem in sources_qa]

In [12]:
response[0]

{'id': '0',
 'source_id': '15592',
 'model': 'gpt-4-0613',
 'temperature': 0.7,
 'labels': [],
 'split': 'train',
 'quality': 'good',
 'response': 'The Anne Frank House has revealed that Anne Frank and her older sister, Margot, likely died at least a month earlier than previously believed. The sisters, who were imprisoned in Nazi concentration camps during the Holocaust, were thought to have died in March 1945, just two weeks before the Bergen-Belsen camp was liberated. However, new research examining archives from the Red Cross, the International Tracing Service, the Bergen-Belsen Memorial, and survivor testimonies suggests that the sisters did not survive until March. The exact dates of their deaths remain unclear, but it is thought that both had symptoms of typhus, the disease they succumbed to, before February 7, 1945.'}

In [13]:
preprocessed_qa = []
model_ids = ["llama-2-7b-chat", "llama-2-13b-chat", "mistral-7B-instruct"]
for elem in response:
    if (not elem["source_id"] in qa_ids) or (not elem["model"] in model_ids):
        continue
    preprocessed_qa.append(elem | {"prompt": id_to_source[elem["source_id"]]["prompt"]})

In [14]:
df = pd.DataFrame(preprocessed_qa)

In [16]:
df.to_csv("assets/data/rag_truth/qa_samples.csv")

In [17]:
preprocessed_summary = []
model_ids = ["llama-2-7b-chat", "llama-2-13b-chat", "mistral-7B-instruct"]
for elem in response:
    if (not elem["source_id"] in summary_cnn_ids) or (not elem["model"] in model_ids):
        continue
    preprocessed_summary.append(elem | {"prompt": id_to_source[elem["source_id"]]["prompt"]})

In [18]:
df = pd.DataFrame(preprocessed_summary)

In [19]:
df.to_csv("assets/data/rag_truth/summ_samples.csv")